<font size="6"> **Statistical Risk Model** </font>

$$ \vec{r}(N,T) = \vec{\beta}(N,K) · \vec{f}(K,T) + \vec{s}(N,T)$$

* from $\vec{f}$ compute `factor covariance matrix`: $\vec{F}$
* from $\vec{s}$ compute `idiosyncratic covariance matrix` $\vec{S}$ and extract diagonal

Use  $\vec{\beta}$, $\vec{F}$ and $\vec{S}$ to estimate portfolio risk: $\sigma_{r_{p}}$

In [1]:
%%capture
%run 01_portopt_data.ipynb

In [2]:
import matplotlib.pyplot as plt
import scipy
import pickle
import datetime as dt

from mle_quant_utils import portopt_utils


%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

In [3]:
import yaml
import os

# Retrieve parameters from configuration file
with open("../conf.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile)

In [4]:
RND_SEED = 123

In [5]:
pd.concat([daily_returns.head(2), daily_returns.tail(2)],axis=0)

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
2015-05-28 00:00:00+00:00,-0.004683,-0.006383,-0.005534,-0.002000,0.001341,-0.004308,0.005088,0.001448,-0.001809,0.000284,...,-0.004758,0.002820,0.000000,0.005400,0.006105,0.005479,-0.001949,-0.007701,-0.004797,-0.006864
2015-05-29 00:00:00+00:00,-0.012719,0.009538,-0.020019,-0.011390,-0.013042,-0.004782,-0.013383,-0.008980,-0.011560,-0.005119,...,0.001594,-0.007940,0.001050,-0.002207,-0.006944,-0.003519,-0.018412,0.006169,-0.008223,0.013427
2015-12-24 00:00:00+00:00,-0.003682,0.012022,0.000465,-0.005341,-0.002041,0.000090,0.000000,-0.001820,-0.004224,0.005673,...,0.009623,-0.000620,-0.010724,-0.002127,0.005553,-0.001614,-0.001620,0.001364,0.003975,0.003121
2015-12-28 00:00:00+00:00,0.007040,-0.013259,0.009526,-0.011204,0.004953,0.002309,-0.001550,-0.001441,-0.001060,-0.006164,...,0.000503,-0.001064,-0.007439,0.004930,-0.021138,-0.003484,-0.002177,-0.006413,-0.005033,-0.004784


In [6]:
def get_total_transaction_costs(h0, h_star, Lambda):
    """
    Calculate Total Transaction Costs

    Parameters
    ----------
    h0 : Pandas Series
        initial holdings (before optimization)
        
    h_star: Numpy ndarray 
        optimized holdings
        
    Lambda : Pandas Series  
        Lambda
        
    Returns
    -------
    total_transaction_costs : float
        Total Transaction Costs
    """
    
    return np.dot(Lambda, (h_star-h0)**2)

In [7]:
risk_aversion = 1.0e-6

def get_obj_func(h0, risk_aversion, Q, specVar, alpha_vec, Lambda): 
    def obj_func(h):
        _h0 = h0.values
        factor_risk = 0.5*risk_aversion*np.sum( np.matmul(Q,h)**2 )
        idiosyncratic_risk = 0.5*risk_aversion*np.dot(h**2, specVar)  # specVar is diag
        exp_port_ret = np.dot(h, alpha_vec)
        trans_costs = np.dot((h-_h0)**2, Lambda)
        obj_f = factor_risk + idiosyncratic_risk - exp_port_ret + trans_costs
        return(obj_f)
    
    return obj_func

In [8]:
def get_grad_func(h0, risk_aversion, Q, QT, specVar, alpha_vec, Lambda):
    def grad_func(h):
        
        grad_factor_risk = risk_aversion*np.matmul(QT, np.matmul(Q,h))
        grad_idiosyncratic_risk = risk_aversion*specVar*h
        grad_exp_port_ret =  alpha_vec
        grad_trans_costs = 2*(h-h0)*Lambda
        
        g = grad_factor_risk + grad_idiosyncratic_risk - grad_exp_port_ret + grad_trans_costs
        
        return(np.asarray(g))
    
    return grad_func

In [9]:
def get_h_star(risk_aversion, Q, QT, specVar, alpha_vec, h0, Lambda):
    """
    Optimize the objective function

    Parameters
    ----------        
    risk_aversion : int or float 
        Trader's risk aversion
        
    Q : patsy.design_info.DesignMatrix 
        Q Matrix
        
    QT : patsy.design_info.DesignMatrix 
        Transpose of the Q Matrix
        
    specVar: Pandas Series 
        Specific Variance
        
    alpha_vec: patsy.design_info.DesignMatrix 
        alpha vector
        
    h0 : Pandas Series  
        initial holdings
        
    Lambda : Pandas Series  
        Lambda
        
    Returns
    -------
    optimizer_result[0]: Numpy ndarray 
        optimized holdings
    """
    obj_func = get_obj_func(h0, risk_aversion, Q, specVar, alpha_vec, Lambda)
    grad_func = get_grad_func(h0, risk_aversion, Q, QT, specVar, alpha_vec, Lambda)


    optimizer_result = scipy.optimize.fmin_l_bfgs_b(
        func=obj_func, x0=h0, fprime=grad_func
    )
    return optimizer_result[0]

In [10]:
B = risk_model['factor_betas']
B.shape

(490, 20)

In [11]:
Fvar = np.diag(risk_model['factor_var_vector'].values[:,0])
Fvar.shape

(20, 20)

In [12]:
adv.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
2015-05-28 00:00:00+00:00,4.111406e+06,1.211964e+07,1.116696e+06,5.178314e+07,1.166691e+07,1.909807e+06,4.556394e+06,2.050929e+06,2.034503e+06,2.451119e+06,...,3.527263e+06,3.064407e+06,1.011783e+07,769575.033333,3.052679e+06,903636.000000,4.765305e+06,1.185707e+06,1.829931e+06,3.423406e+06
2015-05-29 00:00:00+00:00,4.232713e+06,1.232232e+07,1.127231e+06,5.253365e+07,1.165474e+07,2.031373e+06,4.719669e+06,2.043558e+06,2.075111e+06,2.481701e+06,...,3.704804e+06,3.078346e+06,1.028647e+07,780547.933333,3.071932e+06,915696.933333,4.788904e+06,1.212177e+06,1.860690e+06,3.557759e+06
2015-06-01 00:00:00+00:00,4.207142e+06,1.276423e+07,1.126180e+06,5.187218e+07,1.173880e+07,2.000128e+06,4.744443e+06,2.002701e+06,2.022854e+06,2.481818e+06,...,3.728765e+06,3.217187e+06,1.010824e+07,789877.633333,3.063062e+06,902927.533333,4.734382e+06,1.205327e+06,1.926798e+06,3.611460e+06
2015-06-02 00:00:00+00:00,4.267721e+06,1.294121e+07,1.129064e+06,5.142595e+07,1.168395e+07,1.999798e+06,4.803316e+06,1.931299e+06,2.017709e+06,2.493255e+06,...,3.737665e+06,3.276708e+06,1.003273e+07,782557.900000,3.071228e+06,914531.366667,4.708480e+06,1.208697e+06,2.196096e+06,3.604470e+06
2015-06-03 00:00:00+00:00,4.267152e+06,1.302195e+07,1.132370e+06,5.137757e+07,1.209551e+07,2.009004e+06,4.626437e+06,1.910404e+06,2.020291e+06,2.486315e+06,...,3.739011e+06,3.272343e+06,9.968704e+06,775456.533333,3.070423e+06,912325.266667,4.716952e+06,1.205217e+06,2.271745e+06,3.565709e+06


In [13]:
# data alignment

betas_exp = {}
spec_var = {}
lambdas = {}
for dt_idx in daily_returns.index:
    dt_idx_str = dt.datetime.strftime(dt_idx, "%Y-%m-%d")
    universe = ml_alpha.loc[dt_idx].index
    betas_exp[dt_idx_str] = B.loc[universe]
    spec_var[dt_idx_str] = risk_model['idiosyncratic_var_vector'].loc[universe, '0']
    lambdas[dt_idx_str] = adv.loc[dt_idx, universe] * 0.1

In [14]:
alpha_vec = ml_alpha.loc[universe_start_date]
h0 = pd.Series(index=alpha_vec.index, data=np.zeros(len(alpha_vec)))

In [15]:
example_dt = dt.datetime.strftime(universe_start_date, "%Y-%m-%d")
BT = betas_exp[example_dt].T
Q = np.matmul(scipy.linalg.sqrtm(Fvar), BT)
QT = Q.transpose()
specVar = spec_var[example_dt]
Lambda = lambdas[example_dt]

In [18]:

h_star = get_h_star(risk_aversion, Q, QT, specVar, alpha_vec, h0, Lambda)
total_transaction_costs = get_total_transaction_costs(h0, h_star, Lambda)